In [1]:
# Read GlobalLandTemperaturesByMajorCity.csv, in which City names may contain accented character.  
import pandas as pd
import numpy as np
import os.path as path
INPUT_DIR = '../input/'
INPUT_FILE = 'GlobalLandTemperaturesByMajorCity.csv'
temp_raw = pd.read_csv(path.join(INPUT_DIR, INPUT_FILE), encoding='utf-8')
temp_raw.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [2]:
# Round AverageTemperature to 3 decimal places
temp_rounded = temp_raw.round(decimals=3)

In [3]:
# Drop temperature data missing temperature value
temp_complete = temp_rounded.dropna(axis=0, how='any')
# Take 2013 year of data
temp_2013 = temp_complete[temp_complete['dt'] == '2013-01-01']

In [4]:
!pip3 install pymongo

In [5]:
# Persist temperature data in the database
import pymongo
DB_HOST = 'mongo'
DB_PORT = 27017
DB_NAME = 'temperature'
from pymongo import MongoClient
client = MongoClient(DB_HOST, DB_PORT)
db = client[DB_NAME]
posts = db.posts
temp_entries_id = []
temp_2013.apply(lambda x: temp_entries_id.append(x.to_dict()), axis=1)
temp_entries_plain = [dict(x) for x in temp_entries_id]    
posts.delete_many({})
posts.insert_many(temp_entries_id)

In [6]:
!pip3 install rejson

In [7]:
# Store temperature data in the cache
from rejson import Client, Path
from collections import defaultdict
CACHE_HOST = 'redis'
CACHE_PORT = 6379
CACHE_INDEX = 'temperature'
temp_entries_dict = defaultdict(dict)
temp_entries_dict = {x['City']: {k:v for k, v in x.items() if k != '_id'} for x in temp_entries_id}
rj = Client(host=CACHE_HOST, port=CACHE_PORT, decode_responses=True)
rj.jsonset(CACHE_INDEX, Path.rootPath(), temp_entries_dict)
j = rj.jsonget(CACHE_INDEX, Path.rootPath(), no_escape=True)
print(j)

{'Abidjan': {'dt': '2013-01-01', 'AverageTemperature': 27.751, 'AverageTemperatureUncertainty': 0.856, 'City': 'Abidjan', 'Country': "Côte D'Ivoire", 'Latitude': '5.63N', 'Longitude': '3.23W'}, 'Addis Abeba': {'dt': '2013-01-01', 'AverageTemperature': 18.852, 'AverageTemperatureUncertainty': 0.998, 'City': 'Addis Abeba', 'Country': 'Ethiopia', 'Latitude': '8.84N', 'Longitude': '38.11E'}, 'Ahmadabad': {'dt': '2013-01-01', 'AverageTemperature': 19.645, 'AverageTemperatureUncertainty': 0.443, 'City': 'Ahmadabad', 'Country': 'India', 'Latitude': '23.31N', 'Longitude': '72.52E'}, 'Aleppo': {'dt': '2013-01-01', 'AverageTemperature': 6.601, 'AverageTemperatureUncertainty': 0.409, 'City': 'Aleppo', 'Country': 'Syria', 'Latitude': '36.17N', 'Longitude': '37.79E'}, 'Alexandria': {'dt': '2013-01-01', 'AverageTemperature': 15.039, 'AverageTemperatureUncertainty': 0.334, 'City': 'Alexandria', 'Country': 'Egypt', 'Latitude': '31.35N', 'Longitude': '30.16E'}, 'Ankara': {'dt': '2013-01-01', 'AverageTe